In [1]:
# Import necessary libraries
import os  # For file and directory operations
import re  # For regular expressions
import shutil  # For high-level file operations
import csv  # For reading and writing CSV files
import pandas as pd  # For handling Excel files
import openpyxl  # Required for pandas to read Excel files
from argparse import ArgumentParser  # For parsing command-line arguments
import Reorganizationfunction as Rf 


In [2]:
input_path= "/projects/b1042/GoyalLab/aleona/YG10Xbarcode/rawData_YG"
output_path = "/projects/b1042/GoyalLab/aleona/YG10Xbarcode/Experiments "
stagger_file_path= "/projects/b1042/GoyalLab/aleona/YG10Xbarcode/Experiments/StaggerFileAll.csv"
script_path= "/projects/b1042/GoyalLab/aleona/gDNA_Barcode_Analysis"


In [3]:
with open(stagger_file_path, 'r') as f:
    reader = csv.DictReader(f)
    # Trim whitespace from fieldnames
    reader.fieldnames = [field.strip() for field in reader.fieldnames]
    print(f"CSV columns: {reader.fieldnames}")
    
    # Find the correct column name for "Sample" (case-insensitive)
    sample_column = next(
        (field for field in reader.fieldnames if field.lower() == 'sample'), None)
    if sample_column is None:
        raise ValueError(
            f"'Sample' column not found in {stagger_file_path}. Available columns: {', '.join(reader.fieldnames)}")
    
    sample_ids = [row[sample_column] for row in reader]
    print(f"Number of sample IDs read: {len(sample_ids)}")
    print(f"First few sample IDs: {sample_ids[:5]}")

CSV columns: ['Sample', 'Path', 'Index2', 'Stagger_Length']
Number of sample IDs read: 16
First few sample IDs: ['BC01-1-1uM-PLX', 'BC01-2-1uM-PLX', 'BC01-1-Naive', 'BC01-2-Naive', 'BC02-A-1uM-PLX']


In [26]:
import re
from collections import defaultdict

def find_common_prefix(sample_ids):
    if not sample_ids:
        return ""
    
    # Start with the first sample ID as the prefix
    prefix = sample_ids[0]
    
    for sample_id in sample_ids[1:]:
        # Find the common part between the current prefix and the new sample ID
        common = ""
        for i, (c1, c2) in enumerate(zip(prefix, sample_id)):
            if c1 != c2:
                break
            common += c1
        
        # Update the prefix to be the common part
        prefix = common
        
        # If we've reduced the prefix to nothing, stop
        if not prefix:
            break
    
    # Remove any trailing non-alphanumeric characters
    prefix = re.sub(r'[^a-zA-Z0-9]+$', '', prefix)
    
    return prefix


def group_samples(sample_ids):
    # Initialize dictionaries to store groups and reverse mapping
    groups = defaultdict(list)
    sample_to_group = {}
    
    # Find the common prefix among all sample IDs
    common_prefix = find_common_prefix(sample_ids)
    prefix_length = len(common_prefix)
    
    # If common prefix is very short, try to extend it
    if prefix_length < 3:
        pattern = re.compile(rf'^({re.escape(common_prefix)}[A-Za-z0-9-]+)')
    else:
        pattern = re.compile(rf'^({re.escape(common_prefix)}[A-Za-z0-9]*)')
    
    # Group each sample ID and create reverse mapping
    for sample_id in sample_ids:
        match = pattern.match(sample_id)
        if match:
            group = match.group(1)
            groups[group].append(sample_id)
            sample_to_group[sample_id] = group
        else:
            # Fallback: use the whole sample ID as its own group
            groups[sample_id].append(sample_id)
            sample_to_group[sample_id] = sample_id
    
    return groups, sample_to_group

# Function to get sample name from filename
def get_sample_name(filename, group_samples):
    # filename is the name of the file we are trying to match to the samplenames/subfolder
    # group_samples is the name of the samplenames/subfolder

    # Return the first sample name that matches the filename
    # Next returns the second argument, which is None if the first value is empty, or there is no match
    return next((sample for sample in group_samples if sample in filename), None)
    

In [5]:
# Group samples
sample_groups,sample_to_group  = Rf.group_samples(sample_ids)
print(f"Number of groups: {len(sample_groups)}")
for group, samples in list(sample_groups.items())[:5]:
    print(f"Group {group}: {samples}")


print(sample_to_group.keys())

Number of groups: 3
Group BC01: ['BC01-1-1uM-PLX', 'BC01-2-1uM-PLX', 'BC01-1-Naive', 'BC01-2-Naive']
Group BC02: ['BC02-A-1uM-PLX', 'BC02-B-1uM-PLX', 'BC02-A-100nM-PLX-1', 'BC02-A-100nM-PLX-2', 'BC02-B-100nM-PLX']
Group BC03: ['BC03-1-1uM-PLX-1', 'BC03-1-1uM-PLX-2', 'BC03-4-1uM-PLX', 'BC03-2-100nM-PLX', 'BC03-3-100nM-PLX', 'BC03-4-100nM-PLX', 'BC03-Naive']
dict_keys(['BC01-1-1uM-PLX', 'BC01-2-1uM-PLX', 'BC01-1-Naive', 'BC01-2-Naive', 'BC02-A-1uM-PLX', 'BC02-B-1uM-PLX', 'BC02-A-100nM-PLX-1', 'BC02-A-100nM-PLX-2', 'BC02-B-100nM-PLX', 'BC03-1-1uM-PLX-1', 'BC03-1-1uM-PLX-2', 'BC03-4-1uM-PLX', 'BC03-2-100nM-PLX', 'BC03-3-100nM-PLX', 'BC03-4-100nM-PLX', 'BC03-Naive'])


In [6]:
# Initialize variables for file counting and matching
file_count = 0
match_count = 0
main_folders = {}


# Walk through the input directory
for root, _, files in os.walk(input_path):
    # Filter for FASTQ files
    fastq_files = [f for f in files if f.endswith('.fastq.gz')]

print(fastq_files)

NameError: name 'fastq_files' is not defined

In [7]:
"""Main function to reorganize FASTQ files and create necessary CSV files."""

# Print starting message with input parameters
print(f"Starting main function with inputs: {input_path}, {output_path}, {stagger_file_path}, {script_path}")

## Open and read the StaggerFileAll.csv file
with open(stagger_file_path, 'r') as f:
    reader = csv.DictReader(f)
    # Trim whitespace from fieldnames
    reader.fieldnames = [field.strip() for field in reader.fieldnames]

    # Find the correct column name for "Sample" (case-insensitive)
    sample_column = next(
        (field for field in reader.fieldnames if field.lower() == 'sample'), None)

    # Raise an error if the 'Sample' column is not found
    if sample_column is None:
        raise ValueError(
            f"'Sample' column not found in {stagger_file_path}. Available columns: {', '.join(reader.fieldnames)}")

    # Extract sample IDs from the 'Sample' column
    sample_ids = [row[sample_column] for row in reader]


# Group the samples and get the reverse mapping function
sample_groups, sample_to_group = Rf.group_samples(sample_ids)

# Print the number of groups
print(f"Number of groups: {len(sample_groups)}")

# Print the first 5 groups and their samples
for group, samples in list(sample_groups.items())[:5]:
    print(f"Group {group}: {samples}")

# Initialize variables for file counting and matching
file_count = 0
match_count = 0
main_folders = defaultdict(list)  # Use defaultdict instead of regular dict


# Walk through the input directory
for root, _, files in os.walk(input_path):
    # Filter for .fastq.gz files
    # Find if in that folder sets there are files of the format
    fastq_files = [f for f in files if f.endswith('.fastq.gz')]
    
    # Process each fastq file
    for file in fastq_files:
        # Get the full file path
        file_path = os.path.join(root, file)
        
        # Find the matching sample and group
        # sample_to_group.keys() is the subfolder under raw 
        # sample_to_group.values() is the mainfolder
        sample_name = Rf.get_sample_name(file, sample_to_group.keys())
        
        # If a matching sample is found, process the file
        if sample_name:
            # Get the group for this sample
            group = sample_to_group[sample_name]
            
            # Update counters
            match_count += 1
            
            # Create main folder path and add to dictionary
            main_folder_path = os.path.join(output_path, group)
            main_folders[main_folder_path].append(sample_name)
            
            # Create new directory structure for the sample
            sample_dir = os.path.join(main_folder_path, "raw", sample_name)
            
            # Create the sample directory
            os.makedirs(sample_dir, exist_ok=True)
            
            # Set the destination path for the file
            destination = os.path.join(sample_dir, file)
            
            # Copy the file to its new location
            try:
                shutil.copy2(file_path, destination)
                print(f"Copied {file_path} to {destination}")
                file_count += 1
            except Exception as e:
                print(f"Error copying {file_path}: {e}")
        else:
            # Print a message if no matching sample is found
            print(f"Could not determine sample for file: {file}")

# Print summary of processed files
print(f"Total files processed: {file_count}")
print(f"Found {match_count} matching files")
print(f"Processed {len(main_folders)} main folders")

# Define the source file path for the config template
sourcefile = os.path.join(script_path, "templateInputFullRun.py")

Starting main function with inputs: /projects/b1042/GoyalLab/aleona/YG10Xbarcode/rawData_YG, /projects/b1042/GoyalLab/aleona/YG10Xbarcode/Experiments , /projects/b1042/GoyalLab/aleona/YG10Xbarcode/Experiments/StaggerFileAll.csv, /projects/b1042/GoyalLab/aleona/gDNA_Barcode_Analysis
Number of groups: 3
Group BC01: ['BC01-1-1uM-PLX', 'BC01-2-1uM-PLX', 'BC01-1-Naive', 'BC01-2-Naive']
Group BC02: ['BC02-A-1uM-PLX', 'BC02-B-1uM-PLX', 'BC02-A-100nM-PLX-1', 'BC02-A-100nM-PLX-2', 'BC02-B-100nM-PLX']
Group BC03: ['BC03-1-1uM-PLX-1', 'BC03-1-1uM-PLX-2', 'BC03-4-1uM-PLX', 'BC03-2-100nM-PLX', 'BC03-3-100nM-PLX', 'BC03-4-100nM-PLX', 'BC03-Naive']


NameError: name 'defaultdict' is not defined

In [ ]:
import os
import re

def modify_config(source_file, destination_file, new_path_experiment_folder, 
                  new_stagger_file, new_sample_array):
    try:
        # Check if the source file exists
        if not os.path.exists(source_file):
            raise FileNotFoundError(f"Source file not found: {source_file}")

        # Read the content of the source file
        with open(source_file, 'r') as file:
            content = file.read()

        print("Content of the source file:")
        print(content)
        print("\n" + "="*50 + "\n")

        # Define the variables we want to change and their new values
        variables_to_change = {
            'pathExperimentFolder': new_path_experiment_folder,
            'staggerFile': new_stagger_file,
            'sampleArray': new_sample_array,
            'sampleArrayStarcode': new_sample_array
        }

        # Function to replace a variable's value
        def replace_variable(match):
            var_name = match.group(1)
            if var_name in variables_to_change:
                new_value = variables_to_change[var_name]
                if isinstance(new_value, list):
                    return f"{var_name} = {new_value}"
                else:
                    return f"{var_name} = '{new_value}'"
            return match.group(0)

        # Use regex to find and replace variable assignments
        new_content = re.sub(r'(\w+)\s*=\s*["\']?.*?["\']?(?=\n|\Z)', replace_variable, content, flags=re.DOTALL)

        print("Modified content:")
        print(new_content)
        print("\n" + "="*50 + "\n")

        # Ensure the directory for the destination file exists
        os.makedirs(os.path.dirname(destination_file), exist_ok=True)

        # Write the updated content to the destination file
        with open(destination_file, 'w') as file:
            file.write(new_content)

        print(f"Configuration updated and saved to: {destination_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

sourcefile="/projects/b1042/GoyalLab/aleona/gDNA_Barcode_Analysis/templateInputFullRun.py"
destinationfile = "/projects/b1042/GoyalLab/aleona/YG10Xbarcode/Experiments/BC01/templateInputFullRun.py"
main_folder_path = "/projects/b1042/GoyalLab/aleona/YG10Xbarcode/Experiments/BC01"
staggerfilepath= "/projects/b1042/GoyalLab/aleona/YG10Xbarcode/Experiments/BC01/StaggerFile.csv"
sample_id_list

# In your main function or wherever you call modify_config:
try:
    modify_config(sourcefile, destinationfile, main_folder_path, staggerfilepath, sample_id_list)
except Exception as e:
    print(f"Failed to create config file: {destinationfile}")
    print(f"Error: {e}")

In [27]:
sourcefile="/projects/b1042/GoyalLab/aleona/gDNA_Barcode_Analysis/templateInputFullRun.py"
new_path_experiment_folder = "/projects/b1042/GoyalLab/aleona/YG10Xbarcode/Experiments/BC01/templateInputFullRun.py"
main_folder_path = "/projects/b1042/GoyalLab/aleona/YG10Xbarcode/Experiments/BC01"
new_stagger_file= "/projects/b1042/GoyalLab/aleona/YG10Xbarcode/Experiments/BC01/StaggerFile.csv"
new_sample_array = sample_groups.values()

source_file = "/projects/b1042/GoyalLab/aleona/gDNA_Barcode_Analysis/templateInputFullRun.py"


with open(source_file, 'r') as current_file:
    content = current_file.read()

print(f"Original content:\n{content}\n")

# Function to replace a variable assignment
def replace_var(var_name, new_value):
    pattern = f"({var_name}\\s*=\\s*).*"
    replacement = f"\\1{new_value}"
    print(f"Attempting to replace {var_name}")
    print(f"Pattern: {pattern}")
    print(f"Replacement: {replacement}")
    new_content, count = re.subn(pattern, replacement, content, flags=re.MULTILINE)
    if count > 0:
        print(f"Changed {var_name}")
    else:
        print(f"Did not find or change {var_name}")
    print(f"Content after attempted change:\n{new_content}\n")
    return new_content

# Perform replacements
content = replace_var('pathExperimentFolder', f'"{new_path_experiment_folder}"')
content = replace_var('staggerFile', f'"{new_stagger_file}"')
content = replace_var('sampleArray', str(new_sample_array))
content = replace_var('sampleArrayStarcode', str(new_sample_array))

print(content)

Original content: 82 lines
Did not find or change pathExperimentFolder
Did not find or change staggerFile
Changed sampleArray
Changed sampleArrayStarcode
import os
import subprocess
#This file contains all the modifiable variables in this analysis pipeline and no other files need to be modified for regular use. Running this file will call all the steps for the analysis. The file structure of the script must be retained to run this pipeline easily. The description for each parameters is in the documentation doc.

parameters = {
    "pathScript": "path/to/folder/containing/this/file",
    "pathExperimentFolder": "path/to/dataset",

    "step1ExtractBarcode": {
        "staggerFile": "path/to/stagger/file.csv",
        "checkVector": "before",
        "barcodeLength": "90",
        "minPhred": "14",
        "excludedReads": "False", 
        "asciioffset": "33"
    },

    "step2LvHistogramMultipleSamples": {
        "sampleArray": ["S1", "S2", "S3"], 
        "lvHistogramFraction": "part

In [7]:

input_dir = {
    #Input and output directories 
    #Path to Step0_2
    "pathScript": "/projects/b1042/GoyalLab/aleona/gDNA_Barcode_Analysis/Step0_2",
    #The path to the downloaded FASTQ files
    "path":"/projects/b1042/GoyalLab/aleona/gDNA_Barcode_Analysis/SubiaBarcodeseq01-386089854/20230823_Goyal_Subia_Barcodeseq-688734047",
    #The path to sample sheet, if doesn't exist put "NA"
    "sample_sheet":"/projects/b1042/GoyalLab/aleona/gDNA_Barcode_Analysis/Experiment/SAUpdated_GoyalPool3_SampleSheet_Apr2023.xlsx", #created or taken from ENSEMBLE/GENECODE 
    #The path to primer sheet, if doesn't need put "NA"
    "primers_sheet": "/projects/b1042/GoyalLab/aleona/gDNA_Barcode_Analysis/Experiment/20200811_10X_BCSeq_Primers.xlsx",
    # If stagger file all exists already without the need for Sample Sheet or Primers Sheet put the path, if not place it as "NA"
    "staggerfileall": "/projects/b1042/GoyalLab/aleona/YG10Xbarcode/Exp2/StaggerFileAll.csv", # Can be NA
    #The path to Experiments folders
    "exp": "/projects/b1042/GoyalLab/aleona/YG10Xbarcode/Exp2",
    #Depth of subfolders within the Experiment folder above to run the job scripts for
    "depth" : "1" #Default is 1 and usually its the same if the format is followed
}
staggerfileall = input_dir['staggerfileall']
sample_sheet = input_dir['sample_sheet']
primers_sheet = input_dir['primers_sheet']

if staggerfileall == "NA": 
    print("yay")
elif sample_sheet == "NA" and primers_sheet == "NA": 
    print("nay")
